In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
import os                                              
import json
import numpy as np

# === Settings ===
image_size = (128, 128)
batch_size = 2
epochs = 18
num_classes = 7
dataset_dir = "theDataset"

# === Load datasets ===
train_datagen = ImageDataGenerator(rescale=1.0/255)
val_datagen   = ImageDataGenerator(rescale=1.0/255)
test_datagen  = ImageDataGenerator(rescale=1.0/255)

train_gen = train_datagen.flow_from_directory(
    os.path.join(dataset_dir, "Train"),
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical"  # since you're using softmax and 7 classes
)

print(train_gen.class_indices)
# {'angle_1': 0, 'angle_2': 1, ..., 'angle_6': 5, 'noWave': 6}

from collections import Counter

# === Step 1: Compute class counts
counter = Counter(train_gen.classes)
total = sum(counter.values())

# === Step 2: Initial inverse-frequency weighting
class_weight = {i: total / (len(counter) * count) for i, count in counter.items()}

# === Step 3: Boost wave classes
wave_boost_factor = 1.0
wave_class_indices = [0, 1, 2, 3, 4, 5]  # angle_1 to angle_6
for i in wave_class_indices:
    class_weight[i] *= wave_boost_factor

print("📊 Final class weights:", class_weight)


val_gen = val_datagen.flow_from_directory(
    os.path.join(dataset_dir, "Validation"),
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical"
)

test_gen = test_datagen.flow_from_directory(
    os.path.join(dataset_dir, "Test"),
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False  # keep test order stable
)






class FullDatasetPredictionLogger(tf.keras.callbacks.Callback):
    def __init__(self, train_gen, val_gen, test_gen, log_dir="PredictionLogs"):
        super().__init__()
        self.train_gen = train_gen
        self.val_gen = val_gen
        self.test_gen = test_gen
        self.log_dir = log_dir
        os.makedirs(log_dir, exist_ok=True)

        self.logs = {
            "Train": [],
            "Validation": [],
            "Test": []
        }

    def _log_predictions_for_dataset(self, dataset_name, generator, epoch):
        # Predict for the entire dataset
        probs = self.model.predict(generator, verbose=0)
        true_labels = np.argmax(np.vstack([generator[i][1] for i in range(len(generator))]), axis=1)

        self.logs[dataset_name].append({
            "epoch": epoch + 1,
            "predictions": [
                {
                    "true_label": int(true_labels[i]),
                    "probs": probs[i].tolist()
                }
                for i in range(len(probs))
            ]
        })

        # Save to disk
        with open(os.path.join(self.log_dir, f"{dataset_name}_predictions.json"), 'w') as f:
            json.dump(self.logs[dataset_name], f, indent=2)

    def on_epoch_end(self, epoch, logs=None):
        self._log_predictions_for_dataset("Train", self.train_gen, epoch)
        self._log_predictions_for_dataset("Validation", self.val_gen, epoch)
        self._log_predictions_for_dataset("Test", self.test_gen, epoch)

     

import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=[128,128,3]), 
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu',  kernel_initializer='he_uniform', padding='same'), 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu',  kernel_initializer='he_uniform', padding='same'), 
    tf.keras.layers.BatchNormalization(),          
    tf.keras.layers.MaxPooling2D((2,2)),            
    tf.keras.layers.Dropout(0.2),                    
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu',  kernel_initializer='he_uniform', padding='same'), 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu',  kernel_initializer='he_uniform', padding='same'), 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2,2)), 
    tf.keras.layers.Dropout(0.3), 
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(128, activation = 'relu',  kernel_initializer='he_uniform'), 
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Dropout(0.5), 
    tf.keras.layers.Dense(7, activation = 'softmax')
])




# === Compile the model ===
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback

es = EarlyStopping(monitor = "val_accuracy", min_delta = 0.01, patience = 10, verbose = 1)
class CustomModelCheckpoint(Callback):
    def __init__(self, filepath, save_freq):
        super(CustomModelCheckpoint, self).__init__()
        self.filepath = filepath
        self.save_freq = save_freq
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:  # Save on specific iterations (1-indexed)
            self.model.save(self.filepath.format(epoch=epoch + 1))

model_cp = ModelCheckpoint(filepath = 'Keras/gesture_Best.keras', monitor = "val_accuracy",
                           save_best_only = True,
                           save_weights_only = False,
                           verbose = 1)
# Define your custom checkpoint for specific iterations
specific_iteration_cp = CustomModelCheckpoint(filepath='Keras/gesture_Epoch{epoch:02d}.keras',
                                              save_freq=18)  

class HistorySaver(tf.keras.callbacks.Callback):
    def __init__(self, file_path):
        super().__init__()
        self.file_path = file_path
        self.history = []
        self.last_epoch = 0

        # Load existing history if file exists
        if os.path.exists(self.file_path):
            with open(self.file_path, 'r') as file:
                data = json.load(file)
                self.history = data.get('history', [])
                self.last_epoch = data.get('last_epoch', 0)

    def on_epoch_end(self, epoch, logs=None):
        # Append current epoch metrics
        self.history.append({**logs, 'epoch': epoch + 1})  # 1-indexed epoch

        # Save updated history
        with open(self.file_path, 'w') as file:
            json.dump({
                'history': self.history,
                'last_epoch': epoch + 1
            }, file, indent=4)


class SaveHistoryAnd90Acc(Callback):
    def __init__(self, history_path='training_history.json', save_path='gesture_90acc.keras', acc_threshold=0.90):
        super().__init__()
        self.history_path = history_path
        self.save_path = save_path
        self.acc_threshold = acc_threshold
        self.history = []
        self.last_epoch = 0
        self.saved = False  # Tracks if model was already saved at threshold

        # Load existing history if it exists
        if os.path.exists(self.history_path):
            with open(self.history_path, 'r') as file:
                data = json.load(file)
                self.history = data.get('history', [])
                self.last_epoch = data.get('last_epoch', 0)
                
    def on_epoch_end(self, epoch, logs=None):
        # Save training history
        self.history.append({**logs, 'epoch': epoch + 1})
        with open(self.history_path, 'w') as file:
            json.dump({
                'history': self.history,
                'last_epoch': epoch + 1
            }, file, indent=4)

        # Check for 90% training accuracy
        acc = logs.get('accuracy')
        if acc is not None and not self.saved and acc >= self.acc_threshold:
            self.model.save(self.save_path)
            print(f"\n📦 Saved model at epoch {epoch+1} (Training Accuracy: {acc:.4f}) → {self.save_path}")
            self.saved = True
                

class SaveHistoryAnd80Acc(Callback):
    def __init__(self, history_path='training_history.json', save_path='gesture_80acc.keras', acc_threshold=0.80):
        super().__init__()
        self.history_path = history_path
        self.save_path = save_path
        self.acc_threshold = acc_threshold
        self.history = []
        self.last_epoch = 0
        self.saved = False  # Tracks if model was already saved at threshold

        # Load existing history if it exists
        if os.path.exists(self.history_path):
            with open(self.history_path, 'r') as file:
                data = json.load(file)
                self.history = data.get('history', [])
                self.last_epoch = data.get('last_epoch', 0)

    def on_epoch_end(self, epoch, logs=None):
        # Save training history
        self.history.append({**logs, 'epoch': epoch + 1})
        with open(self.history_path, 'w') as file:
            json.dump({
                'history': self.history,
                'last_epoch': epoch + 1
            }, file, indent=4)

        # Check for 80% training accuracy
        acc = logs.get('accuracy')
        if acc is not None and not self.saved and acc >= self.acc_threshold:
            self.model.save(self.save_path)
            print(f"\n📦 Saved model at epoch {epoch+1} (Training Accuracy: {acc:.4f}) → {self.save_path}")
            self.saved = True
class SaveHistoryAnd80Acc(Callback):
    def __init__(self, history_path='training_history.json', save_path='gesture_80acc.keras', acc_threshold=0.80):
        super().__init__()
        self.history_path = history_path
        self.save_path = save_path
        self.acc_threshold = acc_threshold
        self.history = []
        self.last_epoch = 0
        self.saved = False  # Tracks if model was already saved at threshold

        # Load existing history if it exists
        if os.path.exists(self.history_path):
            with open(self.history_path, 'r') as file:
                data = json.load(file)
                self.history = data.get('history', [])
                self.last_epoch = data.get('last_epoch', 0)
    
    def on_epoch_end(self, epoch, logs=None):
        # Save training history
        self.history.append({**logs, 'epoch': epoch + 1})
        with open(self.history_path, 'w') as file:
            json.dump({
                'history': self.history,
                'last_epoch': epoch + 1
            }, file, indent=4)

        # Check for 80% training accuracy
        acc = logs.get('accuracy')
        if acc is not None and not self.saved and acc >= self.acc_threshold:
            self.model.save(self.save_path)
            print(f"\n📦 Saved model at epoch {epoch+1} (Training Accuracy: {acc:.4f}) → {self.save_path}")
            self.saved = True
class SaveHistoryAnd70Acc(Callback):
    def __init__(self, history_path='training_history.json', save_path='gesture_70acc.keras', acc_threshold=0.70):
        super().__init__()
        self.history_path = history_path
        self.save_path = save_path
        self.acc_threshold = acc_threshold
        self.history = []
        self.last_epoch = 0
        self.saved = False  # Tracks if model was already saved at threshold

        # Load existing history if it exists
        if os.path.exists(self.history_path):
            with open(self.history_path, 'r') as file:
                data = json.load(file)
                self.history = data.get('history', [])
                self.last_epoch = data.get('last_epoch', 0)
    
    def on_epoch_end(self, epoch, logs=None):
        # Save training history
        self.history.append({**logs, 'epoch': epoch + 1})
        with open(self.history_path, 'w') as file:
            json.dump({
                'history': self.history,
                'last_epoch': epoch + 1
            }, file, indent=4)

        # Check for 70% training accuracy
        acc = logs.get('accuracy')
        if acc is not None and not self.saved and acc >= self.acc_threshold:
            self.model.save(self.save_path)
            print(f"\n📦 Saved model at epoch {epoch+1} (Training Accuracy: {acc:.4f}) → {self.save_path}")
            self.saved = True

combined_callback = SaveHistoryAnd90Acc(
    history_path='trainingHistory.json',
    save_path='gesture_90acc.keras',
    acc_threshold=0.90
)

combined_callback1 = SaveHistoryAnd80Acc(
    history_path='trainingHistory.json',
    save_path='gesture_80acc.keras',
    acc_threshold=0.80
)

combined_callback2 = SaveHistoryAnd80Acc(
    history_path='trainingHistory.json',
    save_path='gesture_70acc.keras',
    acc_threshold=0.70
)
# Usage
file_path = 'trainingHistory.json'
history_saver = HistorySaver(file_path)
# Load the last completed epoch to start from there
start_epoch = history_saver.last_epoch

prediction_logger = FullDatasetPredictionLogger(train_gen, val_gen, test_gen)
callbacks = [model_cp, specific_iteration_cp, history_saver, combined_callback,combined_callback1,combined_callback2]

model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    class_weight=class_weight,
    callbacks=callbacks,
    verbose=1
)




# === Train the model ===
#history = model.fit(train_gen,callbacks = [model_cp, specific_iteration_cp],validation_data=val_gen,epochs=epochs,class_weight=class_weight, verbose=1)

# === Evaluate on test set ===
test_loss, test_acc = model.evaluate(test_gen, verbose=1)
print(f"\n📊 Final Test Accuracy: {test_acc:.4f}")

# === Save model ===
model.save("gesture_CNN.keras")
print("✅ Model saved as gesture_CNN.keras")


Found 490 images belonging to 7 classes.
{'angle_1': 0, 'angle_2': 1, 'angle_3': 2, 'angle_4': 3, 'angle_5': 4, 'angle_6': 5, 'noWave': 6}
📊 Final class weights: {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0}
Found 105 images belonging to 7 classes.
Found 105 images belonging to 7 classes.


C:\Users\josep\anaconda3\envs\ai_projects-env\lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
C:\Users\josep\anaconda3\envs\ai_projects-env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/18
245/245 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.4066 - loss: 1.7247
Epoch 1: val_accuracy improved from -inf to 0.65714, saving model to Keras/gesture_Best.keras
245/245 ━━━━━━━━━━━━━━━━━━━━ 40s 148ms/step - accuracy: 0.4069 - loss: 1.7234 - val_accuracy: 0.6571 - val_loss: 1.3391
Epoch 2/18
245/245 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.5184 - loss: 1.3679
Epoch 2: val_accuracy improved from 0.65714 to 0.68571, saving model to Keras/gesture_Best.keras
245/245 ━━━━━━━━━━━━━━━━━━━━ 36s 147ms/step - accuracy: 0.5183 - loss: 1.3680 - val_accuracy: 0.6857 - val_loss: 0.7655
Epoch 3/18
245/245 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.5272 - loss: 1.2825
Epoch 3: val_accuracy did not improve from 0.68571
245/245 ━━━━━━━━━━━━━━━━━━━━ 35s 142ms/step - accuracy: 0.5269 - loss: 1.2835 - val_accuracy: 0.0667 - val_loss: 36.9386
Epoch 4/18
245/245 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.2543 - loss: 2.1884
Epoch 4: val_accuracy did not improve from 0.

In [2]:
import os

train_dir = "theDataset/Train"
print("🔍 Subfolders found in Train/:")
print(os.listdir(train_dir))
#checks to see if you have hidden subfolders in your folder

🔍 Subfolders found in Train/:
['angle_1', 'angle_2', 'angle_3', 'angle_4', 'angle_5', 'angle_6', 'noWave']


In [3]:
import shutil
shutil.rmtree("theDataset/Train/.ipynb_checkpoints", ignore_errors=True)
shutil.rmtree("theDataset/Validation/.ipynb_checkpoints", ignore_errors=True)
shutil.rmtree("theDataset/Test/.ipynb_checkpoints", ignore_errors=True)
#If you get errors when trying process your datasets run this block.

In [4]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 128, 128, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 128, 128, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 128, 128, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 64, 64, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 64, 64, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 64, 64, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 64, 64, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 65536)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       8,388,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 25,368,183 (96.77 MB)

 Trainable params: 8,455,847 (32.26 MB)

 Non-trainable params: 640 (2.50 KB)

 Optimizer params: 16,911,696 (64.51 MB)

In [5]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)


0 conv2d
1 batch_normalization
2 conv2d_1
3 batch_normalization_1
4 max_pooling2d
5 dropout
6 conv2d_2
7 batch_normalization_2
8 conv2d_3
9 batch_normalization_3
10 max_pooling2d_1
11 dropout_1
12 flatten
13 dense
14 batch_normalization_4
15 dropout_2
16 dense_1
